In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import EncoderRNN, AttnDecoderRNN
import json
import helpers


encoder_dict = torch.load('./model_v4.pt', map_location=torch.device('cpu'))['encoder_state_dict']
decoder_dict = torch.load('./model_v4.pt', map_location=torch.device('cpu'))['decoder_state_dict']
    
with open('../project_data/project_train_data_instr.json') as json_file:
    train_data = json.load(json_file)

In [2]:
N_EPOCHS = 15
LEARNING_RATE = 0.01
REPORT_EVERY = 1000
HIDDEN_DIM = 256
#BATCH_SIZE = 20
#N_LAYERS = 1
teacher_forcing_ratio = 1
TRAIN_SET_SIZE = 1000
n_words = 43863
MAX_LENGTH = 70

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(10)

encoder = EncoderRNN(n_words, HIDDEN_DIM).to(device)
decoder = AttnDecoderRNN(HIDDEN_DIM, n_words, max_length=MAX_LENGTH).to(device)

encoder.load_state_dict(encoder_dict)
decoder.load_state_dict(decoder_dict)

<All keys matched successfully>

In [3]:
encoder.eval()
decoder.eval()

recipe_step_pairs, idx2word, word2idx, ml = helpers.get_tensor_data()
n_words = len(word2idx)
print(recipe_step_pairs[0])

Number of short ingredient lists:  130567
Average ingredient list length: 14.175872007959267
No ingredients filtered
Max instruction step length:  70
Number of long instructions:  61032
Average instruction length: 149.95270527301457
Total instruction steps:  489828
Recipes filtered:  61455
Recipes left after filtering:  75241
Recipe step pairs:  223824
<SOS> Preheat oven to 400 degrees F ( 205 degrees C ) . Butter a 9x9x2 inch baking pan . <EOS>
<SOS> Melt 1 tablespoon butter in medium nonstick skillet over medium-low heat . Add onion and saute until tender , about 10 minutes . Cool . <EOS>
(tensor([[43860],
        [   17],
        [   18],
        [   19],
        [   20],
        [   21],
        [   22],
        [   23],
        [   24],
        [   21],
        [   25],
        [   26],
        [   27],
        [   28],
        [   29],
        [   30],
        [   31],
        [   32],
        [   33],
        [   27],
        [43862]]), tensor([[43860],
        [   34],
        

In [4]:
from random import choice
from helpers import idx_to_words
from nltk.tokenize import sent_tokenize, word_tokenize
import re

def evaluate(encoder, decoder, input_tensor, gold_standard):
    with torch.no_grad():
        max_length = MAX_LENGTH
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(device)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        loss = 0
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[word2idx['<SOS>']]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if di < len(gold_standard):
                loss += loss_function(decoder_output, gold_standard[di])
            else:
                loss += loss_function(decoder_output, gold_standard[-1])
            if topi.item() == word2idx['<EOS>']:
                if di < len(gold_standard) and gold_standard[di] !=  word2idx['<EOS>']:
                    for dj in range(di, len(gold_standard)):
                        loss += loss_function(decoder_output, gold_standard[dj])
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx2word[str(topi.item())])

            decoder_input = topi.squeeze().detach()

        return decoded_words, loss.item()/len(gold_standard), decoder_attentions

    
def random_evaluate(evaluation_data, n=10):
    for i in range(n):
        pair = choice(evaluation_data)
        print('Instruction step', idx_to_words(pair[0], idx2word))
        print('Next step', idx_to_words(pair[1], idx2word))
        output_words, loss, attentions = evaluate(encoder, decoder, pair[0].to(device), pair[1].to(device))
        output_sentence = ' '.join(output_words)
        print('Generated instructions', output_sentence)
        print("Loss: ", loss)
        print('')
        
        
def evaluate_with_given_input(pair):
    #print('Instruction step', idx_to_words(pair[0], idx2word))
    #print('Next step', idx_to_words(pair[1], idx2word))
    output_words, loss, attentions = evaluate(encoder, decoder, pair[0].to(device), pair[1].to(device))
    output_sentence = ' '.join(output_words)
    #print('Generated instructions', output_sentence)
    return output_sentence, loss, attentions

    
def tokenize(instruction_step):
    words_tokenized = word_tokenize(instruction_step)
    return words_tokenized


def add_helper_tokens(step_tokenized):
    new_step = ['<SOS>']
    new_step.extend(step_tokenized)
    new_step.append('<EOS>')
    return new_step

def to_idx_repr(tokenized_instruction):
    idx_list = [word2idx[w] if w in word2idx else word2idx['<LN>'] for w in tokenized_instruction]
    instr_tensors = torch.tensor(idx_list).view(-1, 1)
    return instr_tensors
    

def prepare_input_instruction(text):
    tokenized = tokenize(text)
    print(tokenized)
    tokenized_h = add_helper_tokens(tokenized)
    tensor = to_idx_repr(tokenized_h)
    return tensor


def prepare_input_instruction_eval(text):
    tokenized = tokenize(text)
    tokenized_h = add_helper_tokens(tokenized[3:-3])
    tensor = to_idx_repr(tokenized_h)
    return tensor


def remove_helper_tokens(text):
    helpers_r = r'(<SOS>)|(<EOS>)'
    cleaned_text = re.sub(helpers_r, "", text, count=2)
    return cleaned_text


def preprocess_instruction_data_from_recipes(recipes, limit):
    preprocessed = []
    filtered_out = 0
    for rec in recipes:
        rec_steps = []
        use_rec = True
        for step in rec:
            if len(step) < limit:
                tensor_step = prepare_input_instruction(step)
                rec_steps.append(tensor_step)
            else:
                use_rec = False
                filtered_out = filtered_out + 1
        if use_rec:
            preprocessed.append(rec_steps)
    print(filtered_out, " recipes filtered out")
    return preprocessed



def generate_next_steps(first_step):
    print('Input: ', first_step)
    steps = []
    made_up_instruction = first_step
    i = 1
    while len(steps) < 10 and made_up_instruction != "<SOS> <EOS>":
        tensor = prepare_input_instruction(made_up_instruction)
        made_up_instruction = evaluate_with_given_input(tensor)
        steps.append(made_up_instruction)
        print(i,".", remove_helper_tokens(made_up_instruction))
        i = i + 1
        

def get_instruction_steps(recipes):
    recipe_step_pairs = []
    for recipe in recipes:
        for i, instr_step in enumerate(recipe[:-1]):
            recipe_step_pairs.append((instr_step, recipe[i+1]))
    print("Recipe step pairs: ", len(recipe_step_pairs))
    return recipe_step_pairs




#made_up_instruction = "chicken Italian-seasoned bread crumbs small onion cloves garlic taste oil Mix ground chicken , 1/4 cup bread crumbs , onion , egg , garlic , salt , and black pepper in a bowl . Moisten hands and shape chicken mixture , 2 tablespoons at a time , into flat , oval-shaped patties ."
#generate_next_steps(made_up_instruction)

loss_function = nn.NLLLoss()
random_evaluate(recipe_step_pairs)

Instruction step <SOS> 3 . Add the broccoli florets , salt and coconut milk , bring to a boil , reduce to medium and cook for 5 minutes . <EOS>
Next step <SOS> 4 . Add the noodles to the coconut milk mixture and toss to thoroughly coat . <EOS>
Generated instructions <SOS> 4 . Pour the mixture into the hot skillet and cook for 5 minutes , or until the onion is translucent . <EOS>
Loss:  11.131607055664062

Instruction step <SOS> In a large bowl , combine the heavy cream and the half and half . Gradually whisk in the sugar until blended . <EOS>
Next step <SOS> Whisk in the vanilla . Refrigerate , covered , until very cold , at least 3 hours or as long as 3 days . <EOS>
Generated instructions <SOS> Add the cream cheese , the cream cheese , and cheese ; mix well . <EOS>
Loss:  9.638291579026442

Instruction step <SOS> Cut each dough ball in half , and cut each half into 12 pieces ( 96 total pieces ) . Working in batches , press each small piece of dough into the bottom and up the sides of 

In [5]:
cookstr = [json.loads(line) for line in open('../../original_data/cookstr-recipes.json', 'r')]

In [6]:
test_recs = [rec['instructions'] for rec in cookstr]
limit = 120
prcessed = preprocess_instruction_data_from_recipes(test_recs, limit)
prcessed = [r for r in prcessed if len(r) > 0]


test_data_steps = get_instruction_steps(prcessed)

['The', 'juice', 'mixture', 'will', 'give', 'the', 'cake', 'a', 'lovely', ',', 'fresh', ',', 'fruity', 'flavor', 'and', 'it', 'is', 'not', 'rich', 'like', 'an', 'icing', '.']
['Leave', 'the', 'cake', 'to', 'cool', '.']
['Preheat', 'a', 'gas', 'grill', 'or', 'prepare', 'a', 'fire', 'in', 'a', 'charcoal', 'grill', '.']
['Serve', 'immediately', 'with', 'a', 'sprinkling', 'of', 'pepper', ',', 'the', 'cilantro', ',', 'a', 'dollop', 'of', 'yogurt', ',', 'and', 'some', 'hunks', 'of', 'bread', '.']
['Cool', 'the', 'génoise', 'layer', '.']
['<', 'p', '>', '<', 'b', '>', 'Serving', '<', '/b', '>', ':', 'Cut', 'diagonal', 'slices', 'to', 'serve.', '<', '/p', '>']
['<', 'p', '>', '<', 'b', '>', 'Storage', '<', '/b', '>', ':', 'Refrigerate', 'the', 'cake', 'until', 'immediately', 'before', 'serving.', '<', '/p', '>']
['To', 'make', 'Scorch', ',', 'omit', 'the', 'tomatoes', 'and', 'bell', 'peppers', 'from', 'the', 'salad', 'and', 'double', 'the', 'amount', 'of', 'habanero', 'and', 'bird', 'chiles', 

['To', 'Make', 'the', 'Ice', 'Cream', ':']
['1', 'Mix', 'pureed', 'bananas', 'into', 'base', 'with', 'an', 'immersion', 'blender', 'or', 'a', 'hand', 'mixer', '.']
['2', 'Process', 'in', 'an', 'ice', 'cream', 'maker', 'according', 'to', 'manufacturer', '’', 's', 'instructions', '.']
['3', 'Meanwhile', ',', 'in', 'a', 'wide', 'saucepan', ',', 'melt', 'butter', 'and', 'brown', 'sugar', 'together', 'and', 'bring', 'to', 'a', 'boil', 'over']
['medium', 'heat', '.', 'Add', 'rum', ',', 'let', 'flame', ',', 'bring', 'back', 'to', 'a', 'boil', ',', 'and', 'turn', 'off', 'heat', '.', 'Let', 'caramel', 'cool', '.']
['4', 'Transfer', 'churned', 'ice', 'cream', 'to', 'a', 'bowl', 'and', 'fold', 'in', 'caramel', 'to', 'make', 'an', 'attractive', 'swirl', '.']
['5', 'Scrape', 'into', 'an', 'airtight', 'storage', 'container', '.', 'Freeze', 'for', 'a', 'minimum', 'of', '2', 'hours', 'before', 'serving', '.']
['To', 'Make', 'the', 'Cookies', ':']
['1', 'Mix', 'wets', ':', 'Place', 'butter', 'in', 'a',

['TO', 'MAKE', 'THE', 'CRUST']
['TO', 'MAKE', 'THE', 'FILLING']
['TO', 'ASSEMBLE', 'THE', 'TART']
['Bake', 'the', 'beans', ',', 'uncovered', ',', '20', 'to', '30', 'minutes', 'or', 'until', 'they', 'are', 'bubbly', 'and', 'the', 'top', 'is', 'crisp', '.']
['To', 'Make', 'the', 'Cake', ':']
['To', 'Make', 'the', 'Creamy', 'Filling', ':']
['Process', 'all', 'the', 'filling', 'ingredients', 'in', 'the', 'food', 'processor', 'until', 'smooth', '.']
['Preheat', 'the', 'oven', 'to', '350°F', '.']
['Variation']
['Boudin', 'Blanc', 'with', 'Apple']
['Blend', 'the', 'butter', 'and', 'flour', 'together', 'in', 'a', 'bowl', 'and', 'then', 'whisk', 'the', 'mixture', 'into', 'the', 'tomato', 'mixture', '.']
['For', 'the', 'bouillabaisse', 'broth', ':']
['For', 'the', 'rouille', ':']
['Rouille', 'should', 'have', 'a', 'consistency', 'similar', 'to', 'mayonnaise', 'thin', 'with', 'a', 'few', 'drops', 'of', 'water', 'or', 'lemon', 'juice', ',', 'if', 'needed', '.']
['For', 'the', 'vegetables', ':']
['

['Let', 'cool', 'before', 'cutting', '.', 'Serve', 'slightly', 'warm', '.']
['Stem', 'and', 'halve', 'the', 'tomatoes', '.', 'In', 'a', 'large', 'bowl', ',', 'combine', 'the', 'ingredients', 'and', 'toss', 'gently', 'to', 'blend', '.', 'Taste', 'for', 'seasoning', '.']
['Preheat', 'the', 'oven', 'to', '225°F', '.']
['Mix', 'gently', ',', 'being', 'careful', 'not', 'to', 'break', 'up', 'the', 'tomatoes', '.', 'Season', 'to', 'taste', 'with', 'salt', 'and', 'pepper', '.']
['Transfer', 'the', 'skewers', 'to', 'a', 'serving', 'platter', 'and', 'season', 'with', 'salt', 'and', 'pepper', '.', 'Serve', 'warm', 'or', 'at', 'room', 'temperature', '.']
['For', 'A', 'Change', ':']
['•', 'Use', 'white', 'wine', 'or', 'balsamic', 'vinegar', 'instead', 'of', 'apple', 'cider', 'vinegar', '.']
['•', 'Cut', 'up', 'a', 'vanilla', 'bean', 'and', 'add', 'it', 'to', 'the', 'jar', '.']
['Meanwhile', ',', 'preheat', 'the', 'oven', 'to', '375°F', '.']
['Brush', 'the', 'bread', 'slices', 'with', 'melted', 'but

['Prepare', 'and', 'bake', 'the', 'Sweet', 'Tart', 'crust', 'or', 'Hazelnut', 'Crust', '.']
['Make', 'the', 'caramel', 'cream', '(', 'see', 'the', 'Note', 'on', 'cooking', 'caramel', 'successfully', ')', ':']
['Emulsify', 'the', 'chocolate', 'with', 'the', 'caramel', 'cream', 'and', 'the', 'eggs', ':']
['Bake', 'the', 'tart', ':']
['Serving', 'Suggestions', ':']
['For', 'making', 'and', 'prebaking', 'the', 'tart', 'shell', ':']
['For', 'the', 'caramel', ':']
['In', 'a', 'small', 'saucepan', ',', 'bring', 'the', '½', 'cup', 'cream', 'to', 'a', 'boil', ',', 'then', 'turn', 'off', 'the', 'heat', '.']
['For', 'the', 'ganache', ':']
['Place', 'the', 'chocolate', 'and', 'salt', 'in', 'a', 'heatproof', 'bowl', '.', 'In', 'a', 'small', 'saucepan', 'over', 'medium', 'heat', ',', 'bring', 'the', 'cream', 'to', 'a', 'boil', '.']
['To', 'Assemble', ':']
['Preheat', 'oven', 'to', '350°F', '(', '180°C', ')', '.']
['In', 'a', 'small', 'bowl', ',', 'stir', 'together', 'graham', 'wafer', 'crumbs', ',',

...

['Serve', 'hot', 'from', 'the', 'griddle', '.']
['Variation', ':', 'Mexi-Stacks']
['1', 'cup', 'grated', 'sharp', 'Cheddar', 'or', 'Monterey', 'Jack', 'cheese']
['Homemade', 'or', 'bottled', 'salsa']
['Dairy', 'or', 'tofu', 'sour', 'cream', '(', 'optional', ')']
['Chopped', 'fresh', 'cilantro', '(', 'optional', ')']
['Roll', 'out', 'the', 'pie', 'dough', 'on', 'a', 'lightly', 'floured', 'work', 'surface', ',', 'and', 'use', 'to', 'line', 'the', 'tart', 'pan', '.', 'Chill', 'for', '30', 'minutes', '.']
['Swirl', 'the', 'Herbsaint', 'around', 'the', 'glass', 'and', 'discard', 'any', 'excess', '.']
['Twist', 'the', 'lemon', 'zest', 'over', 'the', 'top', 'of', 'the', 'liquor', '.']
['Tear', 'the', 'mint', 'leaves', 'into', 'pieces', ',', 'sprinkle', 'over', 'the', 'scafata', ',', 'and', 'serve', '.', 'This', 'dish', 'is', 'also', 'good', 'at', 'room', 'temperature', '.']
['Drizzle', 'the', 'warm', 'sauce', 'over', 'the', 'pancakes', 'and', 'serve', '.']
['Variations', ':']
['•', 'Substitut

['While', 'the', 'ribs', 'are', 'cooking', ',', 'combine', 'the', 'remaining', 'ingredients', 'in', 'a', 'small', 'bowl', '.', 'Stir', 'well', 'and', 'set', 'aside', '.']
['Preheat', 'the', 'oven', 'to', '400°F', '.', 'with', 'the', 'rack', 'on', 'the', 'center', 'shelf', '.']
['In', 'a', 'large', 'non-aluminum', 'kettle', ',', 'combine', 'the', 'peaches', 'with', 'all', 'of', 'the', 'other', 'ingredients', '.']
['Let', 'the', 'peaches', 'cool', 'in', 'their', 'liquid', ',', 'and', 'serve', 'at', 'room', 'temperature', '.']
['Let', 'the', 'peaches', 'cool', 'in', 'their', 'liquid', ',', 'and', 'serve', 'at', 'room', 'temperature', '.']
['In', 'a', 'large', ',', 'heavy', 'pan', ',', 'heat', 'the', 'oil', 'over', 'medium-high', 'heat', '.']
['Add', 'the', 'cumin', ',', 'turmeric', ',', 'cinnamon', ',', 'and', 'cardamom', 'and', 'cook', 'until', 'the', 'seeds', 'sizzle', ',', 'about', '30', 'seconds', '.']
['Add', 'the', 'onion', 'and', 'cook', 'until', 'browned', ',', 'about', '3', 'minu

['Roll', 'out', 'the', 'dough', 'on', 'lightly', 'floured', 'parchment', 'paper', 'to', 'about', '1/8', 'inch', 'thick', '.', 'Chill', 'again', 'for', 'about', '5', 'minutes', '.']
['Keep', 'a', 'tiny', 'ball', 'of', 'excess', 'dough', 'to', 'repair', 'any', 'cracks', 'in', 'the', 'baked', 'shell', '.']
['Bake', 'an', 'additional', '5-10', 'minutes', ',', 'until', 'cooked', 'through', 'and', 'light', 'golden', 'brown', '.']
['Remove', 'the', 'crust', 'from', 'the', 'oven', 'and', 'let', 'it', 'cool', 'for', 'at', 'least', '5', 'minutes', '.']
['Cream', 'the', 'butter', 'and', 'incorporate', 'the', 'eggs', '(', 'see', 'the', 'Notes', 'on', 'creaming', 'butter', 'and', 'room-temperature', 'eggs', ')', ':']
['Roll', 'and', 'blind-bake', 'the', 'tart', 'dough', '(', 'see', 'the', 'Note', 'on', 'rolling', 'tart', 'dough', ')', ':']
['Variation', ':']
['Vanilla-', ',', 'Cardamom-', ',', 'or', 'Cinnamon-Scented', 'Dough', ':']
['Set', 'aside', '.']
['Pour', 'the', 'syrup', 'over', 'the', 'pea

['Variation', ':', 'Pumpkin', 'Cheesecake']
['Add', 'the', 'remaining', 'beans', 'and', 'spinach', 'and', 'simmer', 'for', '30', 'minutes', 'more', '.']
['Place', 'a', 'slice', 'of', 'bread', 'in', 'each', 'soup', 'bowl', '.', 'Ladle', 'in', 'the', 'soup', ',', 'and', ',', 'if', 'you', 'like', ',', 'top', 'with', 'a', 'sprinkling', 'of', 'Parmesan', '.']
['Prep', ':', 'Cut', 'pancetta', 'into', '¼-', 'to', '1½-inch', 'dice', '.', 'Chop', 'onion', '.', 'Chop', 'celery', 'crosswise', 'into', '¼-inch-thick', 'slices', '.']
['Stir', 'in', 'basil', '.', 'Serve', 'hot', '.']
['Variations', ':']
['Pesto', 'vinaigrette', ':']
['Pesto', 'mayonnaise', ':']
['Add', '½', 'cup', 'pesto', 'to', '½', 'cup', 'mayonnaise', '.']
['The', 'Process']
['In', 'a', 'large', 'pot', ',', 'sauté', 'garlic', 'and', 'chili', 'in', 'extra', 'virgin', 'olive', 'oil', '.']
['Add', 'broth', 'and', 'tomatoes', '.']
['Add', 'chicken', '.']
['After', 'about', 'five', 'minutes', 'add', 'potato', '.']
['Cook', 'on', 'a', '

['Chopped', 'red', 'pepper', 'can', 'be', 'used', 'as', 'a', 'garnish', 'for', 'a', 'more', 'colorful', 'combination', 'once', 'they', 'are', 'removed', 'from', 'the', 'broiler', '.']
['To', 'add', 'spice', ',', 'omit', 'basil', 'and', 'use', 'chopped', 'jalapeño', 'and', '1/2', 'cup', 'chopped', 'onion', '.']
['Melba', 'toast', 'rounds', 'can', 'be', 'substituted', 'for', 'zucchini', '.']
['To', 'extract', 'water', 'from', 'the', 'zucchini', ',', 'place', 'it', 'in', 'a', 'colander', 'and', 'press', 'it', 'down', 'against', 'the', 'perforated', 'surface', '.']
['Combine', 'the', 'zucchini', ',', 'eggs', ',', 'onions', ',', 'allspice', ',', 'salt', ',', 'and', 'cinnamon', ',', 'if', 'desired', ',', 'in', 'a', 'medium', 'mixing', 'bowl', '.', 'Mix', 'well', '.']
['Chill', 'for', '2', 'hours', '.']
['The', 'hummus', 'will', 'keep', 'for', '3', 'days', 'in', 'the', 'refrigerator', '.']
['Dry', 'off', 'the', 'bowl', 'and', 'put', 'the', 'zucchini', 'back', 'in', 'it', '.']
['Sift', 'the', 

In [7]:
total_loss = 0
outputs = []

for t in test_data_steps:
    output, loss, attention = evaluate_with_given_input(t)
    total_loss += loss
    outputs.append(output)
    
print("Average loss for test set: ", total_loss/len(test_data_steps))

Average loss for test set:  17.362350101664216


In [8]:
random_evaluate(test_data_steps)

Instruction step <SOS> Toast the waffles to desired crispness . <EOS>
Next step <SOS> Spread the peanut butter on one side of each waffle . <EOS>
Generated instructions <SOS> Place the cooked potatoes on top of the apple mixture and the top of the pastry . <EOS>
Loss:  8.781548720139723

Instruction step <SOS> Add beer ; allow <LN> to continue to ferment for 24 hours more . <EOS>
Next step <SOS> Strain liquid ; discard fruit ; add sugar substitute . <EOS>
Generated instructions <SOS> Stir in the cream cheese , <EOS>
Loss:  6.810345967610677

Instruction step <SOS> Place the mango , juice , and yogurt in a blender , and blend until well combined . <EOS>
Next step <SOS> Pour into individual glasses and serve with a straw . <EOS>
Generated instructions <SOS> Add the cream and mix well . <EOS>
Loss:  9.21328862508138

Instruction step <SOS> Combine the mustard and vinegar in a bowl and stir to mix . Allow the mixture to sit for 15 minutes . <EOS>
Next step <SOS> Place all of the ingredient

In [9]:
evaluate_with_given_input(test_data_steps[205])

('<SOS> Place the bread cubes in a large bowl , and mix in the bread crumbs . <EOS>',
 11.055631510416667,
 tensor([[3.0162e-15, 1.5212e-12, 1.2896e-11,  ..., 6.3481e-11, 8.6996e-11,
          7.9750e-11],
         [1.0734e-16, 1.9691e-10, 1.8610e-12,  ..., 2.5933e-12, 1.3124e-12,
          1.7030e-12],
         [1.0000e+00, 1.3360e-08, 3.7519e-08,  ..., 6.0348e-09, 2.7222e-09,
          5.3871e-09],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]))

In [18]:
from nltk.translate import bleu_score, meteor_score
from nltk.metrics import scores
from rouge_score import rouge_scorer


avg_prec = 0
avg_recall = 0
avg_fscore = 0
avg_bleu = 0
#avg_rouge = []
avg_meteor = 0
avg_len = 0

N = len(test_data_steps)
results = []
targets = []
#scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)


for i, t in enumerate(test_data_steps):
    input_step = [str(s) for s in t[0].flatten().tolist()]
    target = [str(s) for s in t[1].flatten().tolist()]
    targets.append(target)
    result = outputs[i]
    #print(input_step)
    #print(target)
    #print(result)
    result_vec = [str(r) for r in prepare_input_instruction_eval(result).flatten().tolist()]
    results.append(result_vec)
    avg_len += len(result_vec)
    # sanity check
    #prep = idx_to_words(result_vec, idx2word)
    #print(result_vec)
    precision = scores.precision(set(result_vec), set(target))
    avg_prec += precision
    recall = scores.recall(set(result_vec), set(target))
    avg_recall += recall
    f_score = scores.f_measure(set(result_vec), set(target))
    avg_fscore += f_score
    bleu = bleu_score.sentence_bleu([target], result)
    avg_bleu += bleu
    rouge = scorer.score(" ".join(target), " ".join(result))
    #print(rouge)
    #avg_rouge.append(rouge['rougeL']['precision']
    meteor = meteor_score.single_meteor_score(" ".join(target), " ".join(result))
    avg_meteor += meteor

print("Average precision: ", avg_prec/N)
print("Average recall: ", avg_recall/N)
#print("F1-measure: ", avg_fscore/N)

print("Average BLEU: ", avg_bleu/N)
print("Average METEOR: ", avg_meteor/N)
#print("Average ROUGE-L: ", avg_bleu/N)
print("Average step length: ", avg_len/N)

Average precision:  0.38914280386514505
Average recall:  0.3112145435106619
Average BLEU:  8.878445206863794e-234
Average METEOR:  0.0017409686016970072
Average step length:  23.215351812366738


145.95962090650045